# Notebook 7: MCP Server Best Practices

## Best Practices for Agentic AI using MCP

---

**Series:** MCP Server Best Practices - Agentic AI Workflows  
**Notebook:** 7 of 13  
**Level:** Intermediate to Advanced  
**Duration:** ~60 minutes

---

## Learning Objectives

By the end of this notebook, you will:

1. Apply security best practices for MCP servers
2. Implement robust error handling patterns
3. Design servers with clean architecture principles
4. Validate and sanitize all inputs
5. Structure servers for maintainability and testing
6. Follow the principle of least privilege

## 1. The MCP Security Mindset

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP SECURITY PRINCIPLES                               │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   🔐 PRINCIPLE 1: LEAST PRIVILEGE                                       │
│   ─────────────────────────────────                                     │
│   Grant only the minimum permissions needed                              │
│   • Read-only when writes aren't needed                                 │
│   • Specific directories, not entire filesystem                         │
│   • Limited API scopes                                                  │
│                                                                          │
│   🛡️ PRINCIPLE 2: DEFENSE IN DEPTH                                      │
│   ────────────────────────────────────                                  │
│   Multiple layers of security controls                                  │
│   • Input validation at entry                                           │
│   • Path sanitization before operations                                 │
│   • Output encoding before response                                     │
│                                                                          │
│   🔍 PRINCIPLE 3: VALIDATE EVERYTHING                                   │
│   ──────────────────────────────────────                                │
│   Never trust input from any source                                     │
│   • Validate types, ranges, formats                                     │
│   • Sanitize paths and commands                                         │
│   • Escape special characters                                           │
│                                                                          │
│   📝 PRINCIPLE 4: LOG EVERYTHING                                        │
│   ─────────────────────────────────                                     │
│   Comprehensive audit trail                                              │
│   • Log all tool calls with arguments                                   │
│   • Log security denials                                                │
│   • Include correlation IDs                                             │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Setup
import json
from typing import Dict, Any, Optional, List
from dataclasses import dataclass
from enum import Enum

def pprint(obj: dict, title: str = None):
    if title:
        print(f"\n{'='*60}")
        print(f" {title}")
        print(f"{'='*60}")
    print(json.dumps(obj, indent=2, default=str))

## 2. Input Validation Best Practices

### 2.1 Schema-Based Validation with Pydantic

In [ ]:
# Pydantic validation patterns
VALIDATION_CODE = '''
from pydantic import BaseModel, Field, validator, field_validator
from typing import Optional, List
import re


class FilePathInput(BaseModel):
    """Validated file path input."""
    
    path: str = Field(
        ...,
        min_length=1,
        max_length=500,
        description="File path to access"
    )
    
    @field_validator("path")
    @classmethod
    def validate_path(cls, v: str) -> str:
        # Remove null bytes (security risk)
        if "\\x00" in v or "\\0" in v:
            raise ValueError("Path contains null bytes")
        
        # Block common dangerous patterns
        dangerous_patterns = [
            r"\.\.",           # Path traversal
            r"^~",             # Home directory expansion
            r"\$\{",           # Variable expansion
            r"\||",            # Pipe/command injection
            r";|&|`",          # Command chaining
        ]
        
        for pattern in dangerous_patterns:
            if re.search(pattern, v):
                raise ValueError(f"Path contains dangerous pattern")
        
        return v.strip()


class SQLQueryInput(BaseModel):
    """Validated SQL query input - READ ONLY."""
    
    query: str = Field(..., max_length=10000)
    parameters: Optional[dict] = None
    
    @field_validator("query")
    @classmethod
    def validate_query(cls, v: str) -> str:
        # Only allow SELECT statements
        normalized = v.strip().upper()
        
        # Block dangerous keywords
        blocked = ["INSERT", "UPDATE", "DELETE", "DROP", "CREATE", 
                   "ALTER", "TRUNCATE", "EXEC", "EXECUTE", "--"]
        
        for keyword in blocked:
            if keyword in normalized:
                raise ValueError(f"Query contains blocked keyword: {keyword}")
        
        if not normalized.startswith("SELECT"):
            raise ValueError("Only SELECT queries allowed")
        
        return v


class EmailInput(BaseModel):
    """Validated email input."""
    
    to: str = Field(..., pattern=r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\\.[a-zA-Z0-9-.]+$")
    subject: str = Field(..., min_length=1, max_length=200)
    body: str = Field(..., max_length=50000)
    
    @field_validator("subject", "body")
    @classmethod
    def no_null_bytes(cls, v: str) -> str:
        if "\\x00" in v:
            raise ValueError("Content contains null bytes")
        return v
'''

print("Pydantic Validation Patterns:")
print("=" * 60)
print(VALIDATION_CODE)

### 2.2 Using Validators in Tools

In [ ]:
# Applying validators to MCP tools
TOOL_WITH_VALIDATION = '''
from fastmcp import FastMCP
from pydantic import ValidationError

mcp = FastMCP("SecureServer")


@mcp.tool()
def read_file(path: str) -> dict:
    """Read a file with validation."""
    
    # Validate input using Pydantic
    try:
        validated = FilePathInput(path=path)
    except ValidationError as e:
        return {
            "error": "Invalid input",
            "details": e.errors()
        }
    
    # Additional security validation
    # (path allowlist checking, etc.)
    
    # Proceed with validated path
    safe_path = validated.path
    # ... actual file reading logic


@mcp.tool()
def execute_query(query: str, parameters: dict = None) -> dict:
    """Execute a read-only SQL query with validation."""
    
    try:
        validated = SQLQueryInput(query=query, parameters=parameters)
    except ValidationError as e:
        return {
            "error": "Invalid query",
            "details": str(e)
        }
    
    # Execute only validated queries
    # Use parameterized queries to prevent SQL injection
    # ...
'''

print("Tool with Input Validation:")
print("=" * 60)
print(TOOL_WITH_VALIDATION)

## 3. Error Handling Best Practices

### 3.1 The Error Handling Pyramid

In [ ]:
# Error handling patterns
ERROR_HANDLING = '''
import sys
import logging
import traceback
from typing import Callable, TypeVar, Any
from functools import wraps
from enum import Enum

logger = logging.getLogger(__name__)


class ErrorCode(Enum):
    """Standard MCP error codes."""
    PARSE_ERROR = -32700
    INVALID_REQUEST = -32600
    METHOD_NOT_FOUND = -32601
    INVALID_PARAMS = -32602
    INTERNAL_ERROR = -32603
    
    # Custom error codes (use -32000 to -32099)
    SECURITY_ERROR = -32001
    RESOURCE_NOT_FOUND = -32002
    PERMISSION_DENIED = -32003
    RATE_LIMITED = -32004
    TIMEOUT = -32005


class MCPError(Exception):
    """Base exception for MCP errors."""
    
    def __init__(self, code: ErrorCode, message: str, data: Any = None):
        self.code = code
        self.message = message
        self.data = data
        super().__init__(message)
    
    def to_dict(self) -> dict:
        result = {
            "code": self.code.value,
            "message": self.message
        }
        if self.data:
            result["data"] = self.data
        return result


class SecurityError(MCPError):
    """Security-related errors."""
    def __init__(self, message: str, data: Any = None):
        super().__init__(ErrorCode.SECURITY_ERROR, message, data)


class ResourceNotFoundError(MCPError):
    """Resource not found errors."""
    def __init__(self, message: str, data: Any = None):
        super().__init__(ErrorCode.RESOURCE_NOT_FOUND, message, data)


def safe_tool_execution(func: Callable) -> Callable:
    """
    Decorator that wraps tool execution with error handling.
    
    - Catches all exceptions
    - Logs errors appropriately
    - Returns structured error responses
    - Never exposes internal details to client
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
            
        except MCPError as e:
            # Known error - log and return
            logger.warning(f"MCP error in {func.__name__}: {e.message}")
            return {
                "error": e.to_dict(),
                "isError": True
            }
            
        except ValidationError as e:
            # Input validation error
            logger.warning(f"Validation error in {func.__name__}: {e}")
            return {
                "error": {
                    "code": ErrorCode.INVALID_PARAMS.value,
                    "message": "Invalid parameters",
                    "data": str(e)
                },
                "isError": True
            }
            
        except PermissionError as e:
            # System permission error
            logger.error(f"Permission error in {func.__name__}: {e}")
            return {
                "error": {
                    "code": ErrorCode.PERMISSION_DENIED.value,
                    "message": "Permission denied"
                },
                "isError": True
            }
            
        except FileNotFoundError as e:
            # File not found
            logger.warning(f"File not found in {func.__name__}: {e}")
            return {
                "error": {
                    "code": ErrorCode.RESOURCE_NOT_FOUND.value,
                    "message": "Resource not found"
                },
                "isError": True
            }
            
        except Exception as e:
            # Unexpected error - log full trace but don\'t expose
            logger.error(
                f"Unexpected error in {func.__name__}: {e}\\n"
                f"{traceback.format_exc()}"
            )
            return {
                "error": {
                    "code": ErrorCode.INTERNAL_ERROR.value,
                    "message": "Internal server error"
                },
                "isError": True
            }
    
    return wrapper
'''

print("Error Handling Patterns:")
print("=" * 60)
print(ERROR_HANDLING)

### 3.2 Applying Error Handling to Tools

In [ ]:
# Using the error handling decorator
TOOL_WITH_ERROR_HANDLING = '''
from fastmcp import FastMCP

mcp = FastMCP("RobustServer")


@mcp.tool()
@safe_tool_execution
def read_database_record(table: str, id: int) -> dict:
    """
    Read a record from the database.
    
    The @safe_tool_execution decorator ensures:
    - All exceptions are caught
    - Errors are logged appropriately
    - Structured error responses are returned
    - Internal details are never exposed
    """
    # Validate input
    if not table.isalnum():
        raise SecurityError("Invalid table name")
    
    if id < 0:
        raise MCPError(ErrorCode.INVALID_PARAMS, "ID must be positive")
    
    # Attempt operation (might raise various exceptions)
    record = database.get(table, id)
    
    if not record:
        raise ResourceNotFoundError(f"Record {id} not found in {table}")
    
    return {"record": record}
'''

print("Tool with Error Handling:")
print("=" * 60)
print(TOOL_WITH_ERROR_HANDLING)

## 4. Clean Architecture for MCP Servers

### 4.1 Project Structure

In [ ]:
# Recommended project structure
project_structure = """
my-mcp-server/
├── src/
│   ├── __init__.py
│   ├── server.py              # MCP server entry point
│   ├── config.py              # Configuration management
│   │
│   ├── tools/                 # Tool implementations
│   │   ├── __init__.py
│   │   ├── file_tools.py
│   │   ├── database_tools.py
│   │   └── api_tools.py
│   │
│   ├── resources/             # Resource implementations
│   │   ├── __init__.py
│   │   ├── file_resources.py
│   │   └── database_resources.py
│   │
│   ├── prompts/               # Prompt templates
│   │   ├── __init__.py
│   │   └── workflows.py
│   │
│   ├── security/              # Security components
│   │   ├── __init__.py
│   │   ├── validators.py
│   │   ├── auth.py
│   │   └── rate_limiter.py
│   │
│   ├── services/              # Business logic
│   │   ├── __init__.py
│   │   ├── file_service.py
│   │   └── database_service.py
│   │
│   └── utils/                 # Utilities
│       ├── __init__.py
│       ├── logging.py
│       └── errors.py
│
├── tests/                     # Test suite
│   ├── __init__.py
│   ├── test_tools.py
│   ├── test_security.py
│   └── test_integration.py
│
├── pyproject.toml             # Project configuration
├── README.md
└── .env.example               # Environment template
"""

print("Recommended Project Structure:")
print("=" * 60)
print(project_structure)

### 4.2 Separation of Concerns

In [ ]:
# Clean architecture example
CLEAN_ARCHITECTURE = '''
# ═══════════════════════════════════════════════════════════════════════════
# services/file_service.py - Business Logic Layer
# ═══════════════════════════════════════════════════════════════════════════

from pathlib import Path
from typing import Optional, List
from dataclasses import dataclass

@dataclass
class FileInfo:
    """Domain object representing file information."""
    path: str
    name: str
    size: int
    is_directory: bool
    modified: str


class FileService:
    """
    Service layer for file operations.
    
    This layer contains business logic and is independent
    of MCP protocol details.
    """
    
    def __init__(self, path_validator, config):
        self.validator = path_validator
        self.config = config
    
    def list_directory(self, path: str) -> List[FileInfo]:
        """List files in a directory."""
        validated = self.validator.validate_directory(path)
        # ... implementation
    
    def read_file(self, path: str) -> str:
        """Read file contents."""
        validated = self.validator.validate_file_for_read(path)
        # ... implementation
    
    def write_file(self, path: str, content: str) -> FileInfo:
        """Write content to file."""
        validated = self.validator.validate_file_for_write(path)
        # ... implementation


# ═══════════════════════════════════════════════════════════════════════════
# tools/file_tools.py - MCP Tool Layer
# ═══════════════════════════════════════════════════════════════════════════

from fastmcp import FastMCP
from services.file_service import FileService
from utils.errors import safe_tool_execution


def register_file_tools(mcp: FastMCP, file_service: FileService):
    """
    Register file-related tools with the MCP server.
    
    This layer handles MCP protocol specifics and delegates
    to the service layer for business logic.
    """
    
    @mcp.tool()
    @safe_tool_execution
    def list_directory(path: str) -> dict:
        """List files and directories."""
        files = file_service.list_directory(path)
        return {
            "files": [f.__dict__ for f in files]
        }
    
    @mcp.tool()
    @safe_tool_execution
    def read_file(path: str) -> dict:
        """Read file contents."""
        content = file_service.read_file(path)
        return {"content": content}
    
    @mcp.tool()
    @safe_tool_execution
    def write_file(path: str, content: str) -> dict:
        """Write to a file."""
        info = file_service.write_file(path, content)
        return {"success": True, "file": info.__dict__}


# ═══════════════════════════════════════════════════════════════════════════
# server.py - Composition Root
# ═══════════════════════════════════════════════════════════════════════════

from fastmcp import FastMCP
from config import Config
from security.validators import SecurePathValidator
from services.file_service import FileService
from tools.file_tools import register_file_tools


def create_server() -> FastMCP:
    """Factory function to create and configure the MCP server."""
    
    # Load configuration
    config = Config.load()
    
    # Create dependencies
    path_validator = SecurePathValidator(config.allowed_directories)
    file_service = FileService(path_validator, config)
    
    # Create MCP server
    mcp = FastMCP(config.server_name)
    
    # Register tools
    register_file_tools(mcp, file_service)
    
    return mcp


if __name__ == "__main__":
    server = create_server()
    server.run()
'''

print("Clean Architecture Example:")
print("=" * 60)
print(CLEAN_ARCHITECTURE)

## 5. Logging Best Practices

### 5.1 Structured Logging Configuration

In [ ]:
# Logging configuration
LOGGING_CONFIG = '''
import sys
import logging
import json
from datetime import datetime
from typing import Any
import uuid


class StructuredFormatter(logging.Formatter):
    """JSON-structured log formatter."""
    
    def format(self, record: logging.LogRecord) -> str:
        log_data = {
            "timestamp": datetime.utcnow().isoformat() + "Z",
            "level": record.levelname,
            "logger": record.name,
            "message": record.getMessage(),
            "module": record.module,
            "function": record.funcName,
            "line": record.lineno
        }
        
        # Add extra fields
        if hasattr(record, "correlation_id"):
            log_data["correlation_id"] = record.correlation_id
        if hasattr(record, "tool_name"):
            log_data["tool_name"] = record.tool_name
        if hasattr(record, "duration_ms"):
            log_data["duration_ms"] = record.duration_ms
        
        # Add exception info
        if record.exc_info:
            log_data["exception"] = self.formatException(record.exc_info)
        
        return json.dumps(log_data)


def setup_logging(level: str = "INFO"):
    """
    Configure structured logging for MCP server.
    
    CRITICAL: Always log to stderr for STDIO servers!
    """
    
    # Create handler for stderr
    handler = logging.StreamHandler(sys.stderr)
    handler.setFormatter(StructuredFormatter())
    
    # Configure root logger
    root_logger = logging.getLogger()
    root_logger.setLevel(getattr(logging, level.upper()))
    root_logger.addHandler(handler)
    
    # Suppress noisy loggers
    logging.getLogger("httpx").setLevel(logging.WARNING)
    logging.getLogger("httpcore").setLevel(logging.WARNING)


class ToolLogger:
    """Context manager for logging tool executions."""
    
    def __init__(self, tool_name: str, args: dict):
        self.tool_name = tool_name
        self.args = args
        self.correlation_id = str(uuid.uuid4())[:8]
        self.start_time = None
        self.logger = logging.getLogger(f"tools.{tool_name}")
    
    def __enter__(self):
        self.start_time = datetime.now()
        self.logger.info(
            f"Tool invoked",
            extra={
                "correlation_id": self.correlation_id,
                "tool_name": self.tool_name,
                "args": self._sanitize_args(self.args)
            }
        )
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        duration = (datetime.now() - self.start_time).total_seconds() * 1000
        
        if exc_type:
            self.logger.error(
                f"Tool failed: {exc_val}",
                extra={
                    "correlation_id": self.correlation_id,
                    "tool_name": self.tool_name,
                    "duration_ms": duration
                },
                exc_info=True
            )
        else:
            self.logger.info(
                f"Tool completed",
                extra={
                    "correlation_id": self.correlation_id,
                    "tool_name": self.tool_name,
                    "duration_ms": duration
                }
            )
    
    def _sanitize_args(self, args: dict) -> dict:
        """Remove sensitive data from logged arguments."""
        sensitive_keys = {"password", "token", "secret", "key", "auth"}
        return {
            k: "[REDACTED]" if any(s in k.lower() for s in sensitive_keys) else v
            for k, v in args.items()
        }
'''

print("Structured Logging Configuration:")
print("=" * 60)
print(LOGGING_CONFIG)

## 6. Rate Limiting and Resource Protection

In [ ]:
# Rate limiting implementation
RATE_LIMITING = '''
from collections import defaultdict
from datetime import datetime, timedelta
from typing import Optional
import threading


class RateLimiter:
    """
    Simple rate limiter using sliding window.
    
    Protects against:
    - Denial of service attempts
    - Runaway AI loops
    - Resource exhaustion
    """
    
    def __init__(
        self,
        requests_per_minute: int = 60,
        requests_per_hour: int = 1000
    ):
        self.requests_per_minute = requests_per_minute
        self.requests_per_hour = requests_per_hour
        self.minute_windows = defaultdict(list)
        self.hour_windows = defaultdict(list)
        self._lock = threading.Lock()
    
    def check_rate_limit(self, key: str = "default") -> tuple[bool, Optional[str]]:
        """
        Check if request is within rate limits.
        
        Returns:
            (allowed, error_message)
        """
        now = datetime.now()
        
        with self._lock:
            # Clean old entries
            minute_ago = now - timedelta(minutes=1)
            hour_ago = now - timedelta(hours=1)
            
            self.minute_windows[key] = [
                t for t in self.minute_windows[key] if t > minute_ago
            ]
            self.hour_windows[key] = [
                t for t in self.hour_windows[key] if t > hour_ago
            ]
            
            # Check limits
            if len(self.minute_windows[key]) >= self.requests_per_minute:
                return False, f"Rate limit exceeded: {self.requests_per_minute}/minute"
            
            if len(self.hour_windows[key]) >= self.requests_per_hour:
                return False, f"Rate limit exceeded: {self.requests_per_hour}/hour"
            
            # Record request
            self.minute_windows[key].append(now)
            self.hour_windows[key].append(now)
            
            return True, None


# Integration with tools
rate_limiter = RateLimiter(requests_per_minute=30, requests_per_hour=500)


def rate_limited(func):
    """Decorator to apply rate limiting to tools."""
    @wraps(func)
    def wrapper(*args, **kwargs):
        allowed, error = rate_limiter.check_rate_limit(func.__name__)
        
        if not allowed:
            return {
                "error": {
                    "code": -32004,
                    "message": error
                },
                "isError": True
            }
        
        return func(*args, **kwargs)
    
    return wrapper


# Usage
@mcp.tool()
@rate_limited
@safe_tool_execution
def expensive_operation(data: str) -> dict:
    """An operation that should be rate limited."""
    # ... implementation
'''

print("Rate Limiting Implementation:")
print("=" * 60)
print(RATE_LIMITING)

## 7. Testing Best Practices

In [ ]:
# Testing patterns
TESTING_CODE = '''
import pytest
from unittest.mock import Mock, patch


# ═══════════════════════════════════════════════════════════════════════════
# Unit Tests for Security
# ═══════════════════════════════════════════════════════════════════════════

class TestSecurePathValidator:
    """Tests for path validation security."""
    
    @pytest.fixture
    def validator(self, tmp_path):
        """Create validator with temporary directory."""
        allowed = tmp_path / "allowed"
        allowed.mkdir()
        return SecurePathValidator([str(allowed)])
    
    def test_allows_valid_path(self, validator, tmp_path):
        """Test that valid paths are allowed."""
        valid_file = tmp_path / "allowed" / "test.txt"
        valid_file.touch()
        
        result = validator.validate_path(str(valid_file))
        assert result == valid_file.resolve()
    
    def test_blocks_path_traversal(self, validator, tmp_path):
        """Test that path traversal is blocked."""
        malicious = str(tmp_path / "allowed" / ".." / "secret.txt")
        
        with pytest.raises(PathSecurityError):
            validator.validate_path(malicious)
    
    def test_blocks_outside_directory(self, validator):
        """Test that paths outside allowed dirs are blocked."""
        with pytest.raises(PathSecurityError):
            validator.validate_path("/etc/passwd")
    
    def test_blocks_symlink_escape(self, validator, tmp_path):
        """Test that symlinks pointing outside are blocked."""
        allowed = tmp_path / "allowed"
        symlink = allowed / "escape.txt"
        symlink.symlink_to("/etc/passwd")
        
        with pytest.raises(PathSecurityError):
            validator.validate_path(str(symlink))


# ═══════════════════════════════════════════════════════════════════════════
# Unit Tests for Tools
# ═══════════════════════════════════════════════════════════════════════════

class TestFileTools:
    """Tests for file tools."""
    
    @pytest.fixture
    def file_service(self):
        """Create mock file service."""
        return Mock(spec=FileService)
    
    def test_list_directory_success(self, file_service):
        """Test successful directory listing."""
        file_service.list_directory.return_value = [
            FileInfo(path="/test/a.txt", name="a.txt", size=100, 
                    is_directory=False, modified="2024-01-01")
        ]
        
        result = list_directory("/test")
        
        assert "files" in result
        assert len(result["files"]) == 1
    
    def test_list_directory_security_error(self, file_service):
        """Test that security errors are handled."""
        file_service.list_directory.side_effect = PathSecurityError("Blocked")
        
        result = list_directory("/etc")
        
        assert "error" in result
        assert result["isError"] is True


# ═══════════════════════════════════════════════════════════════════════════
# Integration Tests
# ═══════════════════════════════════════════════════════════════════════════

class TestMCPIntegration:
    """Integration tests using actual MCP protocol."""
    
    @pytest.fixture
    def mcp_server(self, tmp_path):
        """Create test server with temporary directory."""
        from server import create_server
        
        with patch("config.ALLOWED_DIRECTORIES", [str(tmp_path)]):
            return create_server()
    
    @pytest.mark.asyncio
    async def test_full_workflow(self, mcp_server, tmp_path):
        """Test complete workflow through MCP protocol."""
        # Create a test file
        test_file = tmp_path / "test.txt"
        test_file.write_text("Hello, World!")
        
        # Test through MCP client
        async with mcp_server.test_client() as client:
            # List directory
            result = await client.call_tool("list_directory", {
                "path": str(tmp_path)
            })
            assert "test.txt" in str(result)
            
            # Read file
            result = await client.call_tool("read_file", {
                "path": str(test_file)
            })
            assert "Hello, World!" in result["content"]
'''

print("Testing Patterns:")
print("=" * 60)
print(TESTING_CODE)

## 8. Best Practices Summary

```
┌─────────────────────────────────────────────────────────────────────────┐
│                    MCP SERVER BEST PRACTICES CHECKLIST                   │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   SECURITY                                                              │
│   ════════                                                              │
│   □ Validate ALL inputs with Pydantic or similar                        │
│   □ Use allowlists, not blocklists                                      │
│   □ Sanitize paths before filesystem operations                         │
│   □ Implement rate limiting                                             │
│   □ Use parameterized queries for databases                             │
│   □ Never expose internal errors to clients                             │
│                                                                          │
│   ERROR HANDLING                                                        │
│   ══════════════                                                        │
│   □ Define custom error types with codes                                │
│   □ Use error handling decorators consistently                          │
│   □ Log all errors with context                                         │
│   □ Return structured error responses                                   │
│   □ Never crash on unexpected errors                                    │
│                                                                          │
│   ARCHITECTURE                                                          │
│   ════════════                                                          │
│   □ Separate tools, services, and security layers                       │
│   □ Use dependency injection                                            │
│   □ Keep tools thin (delegate to services)                              │
│   □ Make components testable in isolation                               │
│                                                                          │
│   LOGGING                                                               │
│   ═══════                                                               │
│   □ ALWAYS log to stderr for STDIO transport                            │
│   □ Use structured JSON logging                                         │
│   □ Include correlation IDs                                             │
│   □ Redact sensitive data                                               │
│   □ Log tool invocations with timing                                    │
│                                                                          │
│   TESTING                                                               │
│   ═══════                                                               │
│   □ Unit test security validators                                       │
│   □ Test error handling paths                                           │
│   □ Integration test with MCP protocol                                  │
│   □ Test rate limiting behavior                                         │
│                                                                          │
└─────────────────────────────────────────────────────────────────────────┘
```

## 9. Key Takeaways

1. **Security First**: Always validate inputs, use allowlists, implement defense in depth
2. **Robust Error Handling**: Never crash, always return structured errors
3. **Clean Architecture**: Separate concerns for maintainability and testability
4. **Comprehensive Logging**: Structured logs to stderr, redact sensitive data
5. **Rate Limiting**: Protect against abuse and runaway loops
6. **Testing**: Unit test security, integration test protocols

### Next Steps

In **Notebook 8: Agentic Workflow Patterns**, we'll:
- Implement Router, Orchestrator-Workers, and Evaluator-Optimizer patterns
- Build multi-step workflows with MCP
- Handle complex agent coordination

---

## References

- [OWASP Input Validation Cheat Sheet](https://cheatsheetseries.owasp.org/cheatsheets/Input_Validation_Cheat_Sheet.html)
- [Pydantic Documentation](https://docs.pydantic.dev/)
- [Python Logging Best Practices](https://docs.python.org/3/howto/logging.html)
- [MCP Security Guidelines](https://modelcontextprotocol.io/docs/learn/security)

---

**Next Notebook:** [08_Agentic_Workflow_Patterns.ipynb](./08_Agentic_Workflow_Patterns.ipynb)